In [1]:
data_path = Path("/kaggle/input/rsna-intracranial-aneurysm-detection") print("Files and folders inside dataset:") print(os.listdir(data_path))

SyntaxError: invalid syntax (2034501799.py, line 1)

In [ ]:
import pandas as pd

train = pd.read_csv("/kaggle/input/rsna-intracranial-aneurysm-detection/train.csv")

In [ ]:
print("First 5 rows of the dataset:")
display(train.head())

In [ ]:
train.info()

In [ ]:
train.shape

In [ ]:
train.describe()

In [ ]:
print(train['PatientSex'].value_counts())
print(train['Modality'].value_counts())

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(8,5))
sns.histplot(train['PatientAge'], bins=30, kde=True)
plt.title('Age Distribution of Patients')
plt.xlabel('Age')
plt.ylabel('Count')
plt.show()


plt.figure(figsize=(6,4))
sns.countplot(data=train, x='PatientSex', hue='Aneurysm Present')
plt.title('Aneurysm Presence by Gender')
plt.show()


In [ ]:
# Step 5: Correlation Heatmap among Artery Columns
artery_cols = [
    'Left Infraclinoid Internal Carotid Artery', 'Right Infraclinoid Internal Carotid Artery',
    'Left Supraclinoid Internal Carotid Artery', 'Right Supraclinoid Internal Carotid Artery',
    'Left Middle Cerebral Artery', 'Right Middle Cerebral Artery',
    'Anterior Communicating Artery', 'Left Anterior Cerebral Artery', 'Right Anterior Cerebral Artery',
    'Left Posterior Communicating Artery', 'Right Posterior Communicating Artery',
    'Basilar Tip', 'Other Posterior Circulation'
]

plt.figure(figsize=(10,8))
sns.heatmap(train[artery_cols].corr(), annot=True, cmap='coolwarm')
plt.title('Correlation Between Artery Aneurysm Locations')
plt.show()


In [ ]:
train.isnull().sum()


In [ ]:

train.groupby('Aneurysm Present')['PatientAge'].describe()


In [ ]:
plt.figure(figsize=(8,6))
sns.boxplot(x='Aneurysm Present', y='PatientAge', data=train, palette='Set2')
plt.title('Age vs Aneurysm Presence')
plt.xlabel('Aneurysm Present (0 = No, 1 = Yes)')
plt.ylabel('Age')
plt.show()


In [ ]:
gender_vs_aneurysm = pd.crosstab(train['PatientSex'], train['Aneurysm Present'], normalize='index') * 100
print(gender_vs_aneurysm)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(6,4))
sns.countplot(data=train, x='PatientSex', hue='Aneurysm Present', palette='Set2')
plt.title('Gender vs Aneurysm Presence')
plt.xlabel('Gender')
plt.ylabel('Count')
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(8,6))
sns.boxplot(data=train, x='PatientSex', y='PatientAge', hue='Aneurysm Present', palette='coolwarm')
plt.title('Age vs Aneurysm Presence by Sex')
plt.xlabel('Patient Sex')
plt.ylabel('Age')
plt.show()


In [ ]:
plt.figure(figsize=(10,6))
sns.boxplot(data=train, x='Modality', y='PatientAge', hue='Aneurysm Present', palette='viridis')
plt.title('Age vs Modality by Aneurysm Presence')
plt.xlabel('Imaging Modality')
plt.ylabel('Patient Age')
plt.xticks(rotation=45)
plt.show()


In [ ]:
artery_cols = train.columns[4:-1]  # all artery-related columns
artery_means = train.groupby('Aneurysm Present')[artery_cols].mean().T

artery_means.plot(kind='bar', figsize=(12,6), color=['skyblue', 'salmon'])
plt.title('Average Artery Involvement by Aneurysm Presence')
plt.ylabel('Proportion of Cases')
plt.xlabel('Artery')
plt.legend(['No Aneurysm', 'Aneurysm Present'])
plt.xticks(rotation=90)
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Select only numeric columns
numeric_df = train.select_dtypes(include=['int64', 'float64'])

# Compute correlation matrix
corr_matrix = numeric_df.corr()

# Plot heatmap
plt.figure(figsize=(12,8))
sns.heatmap(corr_matrix, annot=False, cmap='coolwarm', linewidths=0.3)
plt.title('Correlation Heatmap')
plt.show()


In [ ]:
corr_with_target = corr_matrix['Aneurysm Present'].sort_values(ascending=False)
print(corr_with_target)


In [ ]:
import os
from pathlib import Path

series_path = Path("/kaggle/input/rsna-intracranial-aneurysm-detection/series")

print("Sample folders/files inside 'series':")
print(os.listdir(series_path)[:10])


In [ ]:
sample_series = series_path / os.listdir(series_path)[0]  # pick the first series
print("Inspecting folder:", sample_series)

# List first few files inside that folder
print(os.listdir(sample_series)[:10])


In [ ]:
import pydicom
import matplotlib.pyplot as plt

# Select one DICOM file from the sample folder
sample_dicom_path = sample_series / os.listdir(sample_series)[0]

# Read the DICOM file
dicom = pydicom.dcmread(sample_dicom_path)

# Print DICOM metadata
print("DICOM metadata:")
print(dicom)

# Display the image
plt.imshow(dicom.pixel_array, cmap='gray')
plt.title("Sample DICOM Image")
plt.axis("off")
plt.show()


In [ ]:
import random

# Get 9 random dicom files from the same series
dicom_files = os.listdir(sample_series)
sample_files = random.sample(dicom_files, 9)

fig, axes = plt.subplots(3, 3, figsize=(10, 10))
for i, f in enumerate(sample_files):
    dicom_path = sample_series / f
    img = pydicom.dcmread(dicom_path).pixel_array
    ax = axes[i // 3, i % 3]
    ax.imshow(img, cmap='gray')
    ax.axis('off')
plt.suptitle("Random MRI Slices from Same Patient", fontsize=16)
plt.show()


In [ ]:
import cv2

# Pick one random file from the series
random_file = random.choice(os.listdir(sample_series))
file_path = sample_series / random_file

# Read image
dicom_img = pydicom.dcmread(file_path)
img = dicom_img.pixel_array

# Plot histogram of pixel intensities
plt.figure(figsize=(8,5))
plt.hist(img.ravel(), bins=100, color='gray')
plt.title("Pixel Intensity Distribution in One MRI Slice")
plt.xlabel("Pixel Intensity")
plt.ylabel("Frequency")
plt.show()

# Display the actual image
plt.figure(figsize=(5,5))
plt.imshow(img, cmap='gray')
plt.axis('off')
plt.title("Sample MRI Slice")
plt.show()


In [ ]:
import pydicom
import numpy as np
import os
from pathlib import Path

# Define your dataset path again
series_path = Path("/kaggle/input/rsna-intracranial-aneurysm-detection/series")

# Select the same sample series folder
sample_series = series_path / os.listdir(series_path)[0]


# Read and sort all DICOM slices by InstanceNumber
dicom_files = []
for f in os.listdir(sample_series):
    ds = pydicom.dcmread(sample_series / f)
    dicom_files.append(ds)
dicom_files = sorted(dicom_files, key=lambda d: getattr(d, "InstanceNumber", 0))

# Stack into 3D volume
volume = np.stack([ds.pixel_array for ds in dicom_files]).astype(np.float32)

# Normalize to [0, 1]
volume = (volume - np.min(volume)) / (np.max(volume) - np.min(volume))

print("Volume restored with shape:", volume.shape)


In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(8,5))
plt.hist(volume.ravel(), bins=100, color='purple')
plt.title("Pixel Intensity Distribution in Full 3D Volume (Normalized)")
plt.xlabel("Normalized Intensity")
plt.ylabel("Frequency")
plt.show()


In [ ]:
import cv2

print("Original volume shape:", volume.shape)

# Let's resize each slice to a smaller shape (for faster computation)
target_size = (128, 128)  # (width, height)
resized_slices = [cv2.resize(slice, target_size) for slice in volume]

resized_volume = np.stack(resized_slices)
print("Resized volume shape:", resized_volume.shape)


In [ ]:
# load_preprocessed.py
import numpy as np

def load_train_data():
    X = np.load('data/preprocessed/X_train.npy')
    y = np.load('data/preprocessed/y_train.npy')
    return X, y

def load_test_data():
    X = np.load('data/preprocessed/X_test.npy')
    y = np.load('data/preprocessed/y_test.npy')
    return X, y


In [ ]:
import cv2

print("Original volume shape:", volume.shape)

# Let's resize each slice to a smaller shape (for faster computation)
target_size = (128, 128)  # (width, height)
resized_slices = [cv2.resize(slice, target_size) for slice in volume]

resized_volume = np.stack(resized_slices)
print("Resized volume shape:", resized_volume.shape)


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12,4))
for i, idx in enumerate([20, 60, 100]):
    plt.subplot(1, 3, i+1)
    plt.imshow(resized_volume[idx], cmap='gray')
    plt.title(f"Slice {idx}")
    plt.axis('off')
plt.show()


In [ ]:
# Normalization
normalized_volume = (resized_volume - resized_volume.min()) / (resized_volume.max() - resized_volume.min())


In [ ]:
# Standardization
standardized_volume = (resized_volume - np.mean(resized_volume)) / np.std(resized_volume)


In [ ]:
import matplotlib.pyplot as plt

# Pick some evenly spaced slice indices
slice_indices = [20, 60, 100]

plt.figure(figsize=(15, 8))

# Show normalized volume slices
for i, idx in enumerate(slice_indices):
    plt.subplot(2, len(slice_indices), i + 1)
    plt.imshow(normalized_volume[idx], cmap='gray')
    plt.title(f"Normalized Slice {idx}")
    plt.axis('off')

# Show standardized volume slices
for i, idx in enumerate(slice_indices):
    plt.subplot(2, len(slice_indices), len(slice_indices) + i + 1)
    plt.imshow(standardized_volume[idx], cmap='gray')
    plt.title(f"Standardized Slice {idx}")
    plt.axis('off')

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
from pathlib import Path

# Create a folder to store preprocessed data
save_path = Path("preprocessed_data")
save_path.mkdir(exist_ok=True)

# Save normalized and standardized volumes
np.save(save_path / "normalized_volume.npy", normalized_volume)
np.save(save_path / "standardized_volume.npy", standardized_volume)

print("✅ Volumes saved successfully!")
print(f"Files saved in: {save_path.resolve()}")


In [ ]:
import pydicom
import numpy as np
import cv2
import os
from pathlib import Path
import matplotlib.pyplot as plt

# Define the patient folder
patient_folder = Path("/kaggle/input/rsna-intracranial-aneurysm-detection/series/1.2.826.0.1.3680043.8.498.96155132589464464482407972219172224468")

# Create a list of all DICOM file paths
dicom_files = sorted([patient_folder / f for f in os.listdir(patient_folder) if f.endswith('.dcm')])

# Initialize list to store processed images
processed_slices = []

# Loop through and process each DICOM slice
for f in dicom_files:
    ds = pydicom.dcmread(f)
    img = ds.pixel_array.astype(np.float32)
    
    # Normalize intensity to [0, 1]
    img = (img - np.min(img)) / (np.max(img) - np.min(img))
    
    # Resize to 128x128 (you can change to 256x256 if you have more memory)
    img_resized = cv2.resize(img, (128, 128))
    
    processed_slices.append(img_resized)

# Convert to numpy array
processed_slices = np.array(processed_slices)

print("Processed slices shape:", processed_slices.shape)
print("Min pixel value:", processed_slices.min())
print("Max pixel value:", processed_slices.max())

# Display sample preprocessed slices
plt.figure(figsize=(10, 8))
for i, slice_idx in enumerate(np.linspace(0, len(processed_slices)-1, 9, dtype=int)):
    plt.subplot(3, 3, i+1)
    plt.imshow(processed_slices[slice_idx], cmap='gray')
    plt.axis('off')
plt.suptitle("Normalized and Resized MRI Slices", fontsize=14)
plt.show()


In [ ]:
import os
import numpy as np
import pydicom
import cv2
from pathlib import Path
from tqdm import tqdm

# Paths
series_path = Path("/kaggle/input/rsna-intracranial-aneurysm-detection/series")
output_dir = Path("preprocessed_data")
output_dir.mkdir(exist_ok=True)

# Target size for resizing
target_shape = (128, 128)

# Function to load, resize, and normalize a DICOM series
def load_and_preprocess_series(folder_path):
    dicom_files = sorted(
        [pydicom.dcmread(folder_path / f) for f in os.listdir(folder_path) if f.endswith(".dcm")],
        key=lambda x: int(x.InstanceNumber)
    )

    # Stack slices to form a 3D volume
    volume = np.stack([ds.pixel_array for ds in dicom_files], axis=0)

    # Resize each slice to 128x128
    resized_volume = np.stack([cv2.resize(slice_, target_shape) for slice_ in volume], axis=0)

    # Normalize (0–1)
    normalized = (resized_volume - np.min(resized_volume)) / (np.max(resized_volume) - np.min(resized_volume) + 1e-8)

    # Standardize (mean=0, std=1)
    standardized = (normalized - np.mean(normalized)) / (np.std(normalized) + 1e-8)
    return standardized

# Loop through all patient folders and preprocess
for patient_folder in tqdm(os.listdir(series_path)):
    folder_path = series_path / patient_folder
    if folder_path.is_dir():
        try:
            volume = load_and_preprocess_series(folder_path)
            np.save(output_dir / f"{patient_folder}.npy", volume)
        except Exception as e:
            print(f"Skipping {patient_folder} due to error: {e}")

print("All available series processed and saved in 'preprocessed_data/'")


In [ ]:
!du -sh preprocessed_data
!ls preprocessed_data | wc -l


In [ ]:
!ls -lh


In [ ]:
!df -h


In [ ]:
!df -i


In [ ]:
import os

output_dir = "/kaggle/working/preprocessed_images"
os.makedirs(output_dir, exist_ok=True)
print("Output directory ready:", output_dir)


In [ ]:
import os
import numpy as np
import pydicom
from tqdm import tqdm
from pathlib import Path
from scipy.ndimage import zoom

series_root = Path("/kaggle/input/rsna-intracranial-aneurysm-detection/series")
output_dir = Path("/kaggle/working/preprocessed_images")
output_dir.mkdir(exist_ok=True)

# List all patient folders
series_list = sorted(os.listdir(series_root))

# Detect already-processed files (if resuming)
done = [f.replace(".npy", "") for f in os.listdir(output_dir)]
remaining = [s for s in series_list if s not in done]

print(f"Already done: {len(done)} | Remaining: {len(remaining)}")

# Target shape (H, W, D)
TARGET_SHAPE = (128, 128, 128)

def preprocess_volume(volume, target_shape=TARGET_SHAPE):
    # Make sure it's 3D
    if volume.ndim == 2:
        volume = volume[np.newaxis, :, :]  # add depth axis
    
    h, w, d = volume.shape
    zoom_factors = (target_shape[0]/h, target_shape[1]/w, target_shape[2]/d)
    resized = zoom(volume, zoom_factors, order=1)  # linear interpolation

    # Normalize to 0-1
    resized = resized.astype(np.float32)
    resized = (resized - resized.min()) / (resized.max() - resized.min() + 1e-8)
    
    # Standardize
    resized = (resized - resized.mean()) / (resized.std() + 1e-8)
    
    return resized

for i, series_id in enumerate(tqdm(remaining)):
    series_path = series_root / series_id
    try:
        # Read all DICOM slices
        slices = []
        for f in sorted(os.listdir(series_path)):
            ds = pydicom.dcmread(series_path / f)
            img = ds.pixel_array.astype(np.float32)
            slices.append(img)

        volume = np.stack(slices) if len(slices) > 1 else slices[0]

        # Preprocess
        standardized = preprocess_volume(volume, target_shape=TARGET_SHAPE)

        # Save
        np.save(output_dir / f"{series_id}.npy", standardized)

       
    except Exception as e:
        print(f"Skipping {series_id} due to error:", e)
